In [3]:
from flood_prediction import notebook_utils

notebook_utils.init()

Package root is '/home/mahdi/flood-prediction'.


In [1]:
import typing
from collections.abc import Callable

import numpy as np
import polars as pl
import statsmodels.stats.weightstats  # type: ignore

from flood_prediction import _type_aliases, dataset, utils


In [ ]:
# Y distribution

In [ ]:
# Feature distribution

In [ ]:
# Correlation feature per Y

In [ ]:
# Correlation matrix

In [ ]:
# Train v test

In [ ]:
def compare_features(
    test: Callable[[np.ndarray, np.ndarray], float] = utils.return_arg(  # noqa: B008
        statsmodels.stats.weightstats.ztest, -1
    ),
) -> pl.LazyFrame:
    """Compare the distributions of each feature."""
    features = typing.get_args(_type_aliases.DatasetFeature)
    train_df = dataset.load("train")
    test_df = dataset.load("test")
    return pl.LazyFrame(
        {
            feature: test(
                utils.column_to_series(train_df, feature).to_numpy(),
                utils.column_to_series(test_df, feature).to_numpy(),
            )
            for feature in features
        }
    )
